In [1]:
from dash import Dash, dcc, html, dcc, callback, Output, Input
import dash_bootstrap_components as dbc
import plotly.express as px
import pandas as pd
import numpy as np
import re
from functools import reduce
app = Dash(__name__)
# styles
colors = {
    'background': '#d9ffb3',
    'text': '#0066cc',
    'title' : '#000066'
}

In [2]:
df_mig = pd.read_excel('..\Data\LinkedIn_DigitalData\public_use-talent-migration.xlsx', sheet_name=['Country Migration', 'Industry Migration', 'Skill Migration'])
df_cm = df_mig['Country Migration']
df_im = df_mig['Industry Migration']
df_sm = df_mig['Skill Migration']
df_country = df_cm[['base_country_code','base_lat','base_long']].drop_duplicates(subset=['base_country_code'])
df_im = pd.merge(df_im, df_country, left_on='country_code', right_on='base_country_code').drop('base_country_code',axis=1)
df_sm = pd.merge(df_sm, df_country, left_on='country_code', right_on='base_country_code').drop('base_country_code',axis=1)
df_im['avg_migration'] = df_im[['net_per_10K_2015','net_per_10K_2016','net_per_10K_2017','net_per_10K_2018','net_per_10K_2019']].mean(axis=1)
df_sm['avg_migration'] = df_sm[['net_per_10K_2015','net_per_10K_2016','net_per_10K_2017','net_per_10K_2018','net_per_10K_2019']].mean(axis=1)
df_im['size'] = np.abs(df_im['avg_migration'])
df_sm['size'] = np.abs(df_sm['avg_migration'])

In [3]:
@app.callback(
    [Output("filter-3","options"),
    Output("geo", "figure")],
    [Input("i-s", "value"),
    Input("filter-3", "value")]
)
def geo_update(isoption, sel):
    options = []
    filtered_df = None
    if isoption == 'industry migration':
        options=[{"label": s, "value": s}
                for s in df_im['industry_name'].dropna().unique()]
        filtered_df = df_im[df_im['industry_name']==sel]
    elif isoption == 'skill migration':
        options=[{"label": s, "value": s}
                for s in df_sm['skill_group_name'].dropna().unique()]
        filtered_df = df_sm[df_sm['skill_group_name']==sel]
    fig = px.scatter_geo(
        data_frame= filtered_df,
        lat="base_lat",
        lon="base_long",
        size="size",
        hover_name="country_name",
        projection='equirectangular',color='avg_migration',
                      color_continuous_scale=[(0, 'red'), (0.5, 'yellow'), (1, 'green')],
                      color_continuous_midpoint=0
    )
    fig.update_layout(
    margin=dict(l=20, r=20, t=10, b=20),  # Adjust margins
    font=dict(color=colors['text']),
    paper_bgcolor=colors['background'],
    coloraxis_colorbar=dict(title='Average inflow per 10K')
    )
    return options,fig

In [4]:
app.layout = html.Div(style={'backgroundColor': colors['background'], 'margin': 10, 'padding': 20}, children=[
    html.H1(
        children='Talent Migration',
        style={
            'textAlign': 'center',
            'color': colors['title']
        }
    ),

     dbc.Row([
                dcc.RadioItems(
                id="i-s",
                inline=True,
                options=["industry migration", "skill migration"],
                value="industry migration",
                    style={
                        "color": colors['text'],  
                        'font-size': '20px'  # set the font family of the dropdown to sans-serif
                    }
            ),
             html.Label("Select from options", style={'color': colors['text']}),
                dcc.Dropdown(
                    id="filter-3",
#                     options=[{"label": s, "value": s}
#                              for s in df_growth['isic_section_name'].dropna().unique()],
                    value=None,
                    style={
                        "width": "75%",  # set the width of the dropdown to 50% of its container
                        "height": "50%",  # set the height of each dropdown item to 40 pixels
                        "backgroundColor": colors['background'],  # set the background color of the dropdown to dark gray
                        "color": "#000",  # set the font color of the dropdown to light gray
                        "fontFamily": "sans-serif",  # set the font family of the dropdown to sans-serif
                        "padding":"20",
                    }
                ),
         dcc.Graph(id="geo")
             
     ])
])

In [5]:
if __name__ == "__main__":
    app.run_server(debug=True)